In [1]:
# import
import json
import time, datetime

import grequests
import requests
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 1000)

D:\develop\python\Anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [22]:
# 读取分类表
df_category = pd.read_excel('资产配置分类表.xlsx',dtype={'基金代码': np.object, '分类ID': np.int64})
# df_category.基金代码.unique()
# 基金名称	基金简称	基金代码	一级分类	二级分类	三级分类	分类ID	市场

In [3]:
# config & cookie

global_name = '康力泉'
global_name = '李淑云'
global_name = '康世海'

klq_header = {'Host': 'danjuanapp.com', 
            'Connection': 'keep-alive', 
            'Pragma': 'no-cache', 
            'Cache-Control': 'no-cache', 
            'Accept': 'application/json, text/plain, */*', 
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36', 
            'elastic-apm-traceparent': '00-cf0b429fe7c9fe2c32aade4f02cf9f12-5da687bdd2493f51-01', 
            'Sec-Fetch-Site': 'same-origin', 
            'Sec-Fetch-Mode': 'cors', 
            'Sec-Fetch-Dest': 'empty', 
            'Referer': 'https://danjuanapp.com/my-money?channel=1300100141', 
            'Accept-Encoding': 'gzip, deflate, br', 
            'Accept-Language': 'zh-CN,zh;q=0.9', 
            'Cookie': 'device_id=web_S1ot72YpL; _ga=GA1.2.311399310.1592537987; gr_user_id=1013b64a-5f30-4891-bae2-7345e93e52f7; thirdparty_token=; thirdparty_source=; thirdparty_nickname=; thirdparty_avatar=; Hm_lvt_b53ede02df3afea0608038748f4c9f36=1595420501; channel=1300100141; Hm_lpvt_b53ede02df3afea0608038748f4c9f36=1596033363; acw_tc=2760824c15988776261422279ee0b32c0909bde44b077e7cf56ba26a2f6772; xq_a_token=01cd7758682a857c480534aafa9ea01f6881ae8a; accesstoken=240020000f5c2f25aeb09b87d557a2edbf0c3d526ed48e771; uid=203274561; u=203274561; refreshtoken=240020000ba36ea4764b5561a4de980db4d1101ec943ba9fb; timestamp=1598877646480'}

lsy_header = { 'Host': 'danjuanapp.com', 
            'Connection': 'keep-alive', 
            'Accept': 'application/json, text/plain, */*', 
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36', 
            'elastic-apm-traceparent': '00-202f7df9c2a9ecacf8c83415b7f244ca-92066da89fd9ce82-01', 
            'Sec-Fetch-Site': 'same-origin', 
            'Sec-Fetch-Mode': 'cors', 
            'Sec-Fetch-Dest': 'empty', 
            'Referer': 'https://danjuanapp.com/', 
            'Accept-Encoding': 'gzip, deflate, br', 
            'Accept-Language': 'zh-CN,zh;q=0.9', 
            'Cookie': 'device_id=web_r1ljD0fdkL; _ga=GA1.2.770250261.1577754217; gr_user_id=935b3292-a6c9-49b1-b521-b25bed96076d; thirdparty_token=; thirdparty_source=; thirdparty_nickname=; thirdparty_avatar=; Hm_lvt_b53ede02df3afea0608038748f4c9f36=1591594020; Hm_lpvt_b53ede02df3afea0608038748f4c9f36=1591686446; channel=1300100141; xq_a_token=01cd7758682a857c480534aafa9ea01f6881ae8a; acw_tc=2760776f15989390442294736ea5fbfa88af0b3a3bcbc26e5dc06b75fcdec8; accesstoken=240020000d5c0d98c8d155dc0f01df258eaa29e64bab7c51f; uid=194897199; u=194897199; refreshtoken=240020000045797513af13fe811715643b6bee2dac3206e7d; timestamp=1598939059944'}

ksh_header = {'Accept': 'application/json, text/plain, */*',
            'Accept-Encoding': 'gzip, deflate, br',
            'Accept-Language': 'zh-CN',
            'Connection': 'Keep-Alive',
            'Cookie': 'device_id=web_ByVDEGK9I; refreshtoken=2400200006324b804363685f8af7b583a4aa6630cbe2596fa; u=576923716; gr_user_id=d53bf9ac-42dd-441c-a1b2-0e9354af61f8; _ga=GA1.2.1116723460.1589351520; Hm_lvt_b53ede02df3afea0608038748f4c9f36=1598515263,1598944615; accesstoken=2400200002c8af89f8f7d20e1768c43d9556931fc6fe8631a; timestamp=1598944631452; uid=576923716; acw_tc=2760775115989446131175658ea9d1f585e9f11b438cb4834301e1e77c3cc3; channel=1300100141; xq_a_token=01cd7758682a857c480534aafa9ea01f6881ae8a; Hm_lpvt_b53ede02df3afea0608038748f4c9f36=1598944615; _gid=GA1.2.924514688.1598944615; _gat_gtag_UA_16079156_7=1',
            'elastic-apm-traceparent': '00-3167e31995e74d7e8cd7c227c55814ce-2f06219780deda2e-01',
            'Host': 'danjuanapp.com',
            'Referer': 'https://danjuanapp.com/my-money?channel=1300100141',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18363'}

headers = klq_header
headers = lsy_header
headers = ksh_header

In [4]:
# 获取交易列表

trade_list_url = u'https://danjuanapp.com/djapi/order/p/list?page=1&size=2000&type=all'
response = requests.get(trade_list_url, headers=headers, verify=False)
trade_list = response.json()['data']['items']
# 包含已撤单、失败、进行中等非必要交易列表
# trade_list

D:\develop\python\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [5]:
# 并发获取所有的交易列表详情

series_trade_list = [pd.Series(x) for x in trade_list]
df_trade_list = pd.DataFrame(series_trade_list)
# 去掉失败、撤单、部分交易成功、交易进行中的交易记录单
df_trade_list = df_trade_list[~(df_trade_list['status_desc'].isin(['交易失败', '已撤单', '部分交易成功','交易进行中']))]
# created_at 换成可读形式 1598332156752
df_trade_list['created_at'] = df_trade_list.created_at.apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x/1000)))
df_trade_list['created_at'] = df_trade_list['created_at'].astype(np.datetime64)
df_trade_list = df_trade_list.sort_values(['created_at'])
df_trade_list = df_trade_list.reset_index(drop=True)
df_trade_list.index.name = 'id'
# ['CSI666', 'CSI1021', 'CSI1019', '003474']
# action ['022', '024', '143', '036']
print(df_trade_list.status_desc.unique(), len(df_trade_list))
df_trade_list.to_excel('01_{0}_trade_list.xlsx'.format(global_name), sheet_name=f'{global_name}_交易记录')

['交易成功'] 190


D:\develop\python\Anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [6]:
# 取回所有的成交记录
trade_detail_url = u'https://danjuanapp.com/djapi/order/p/plan/{0}'
# trade_list 唯一 id 集合
order_id_list = df_trade_list.order_id.tolist()
tasks = [grequests.get(u'https://danjuanapp.com/djapi/order/p/plan/{0}'.format(x), headers=headers) for x in order_id_list]
response_list = grequests.map(tasks)
print(len(response_list))
response_list

190


[<Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [

In [23]:
# 汇总所有的成交记录

# 组合交易
plan_record_list = []
# 单独基金交易
fund_record_list = []

for i in range(len(response_list)):
    response = response_list[i]
    if response.status_code != 200:
        print('[Error]: {0} 请求失败'.format(response.request.url))
        exit(1)
    records_data = response.json()['data']
    # 去掉 单纯买入南方天天利货币B 的操作，这种操作 ttype 是 fund，sub_order_list 是空
    if 'sub_order_list' not in records_data.keys():
        fund_record_list.append(pd.Series(response.json()['data']))
    else:
        plan_record_list.append(pd.Series(response.json()['data']))
#     break
df_plan_record_list = pd.DataFrame(plan_record_list)
df_fund_record_list = pd.DataFrame(fund_record_list)

# 单独基金交易
df_fund_record_list['created_at'] = df_fund_record_list.created_at.apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x/1000)))
df_fund_record_list['created_at'] = df_fund_record_list['created_at'].astype(np.datetime64)
df_fund_record_list = df_fund_record_list.sort_values(['created_at'])
df_fund_record_list.to_excel('02_{0}_fund_order_list.xlsx'.format(global_name), sheet_name=f'{global_name}_交易记录')

# 组合操作
df_plan_record_list['created_at'] = df_plan_record_list.created_at.apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x/1000)))
df_plan_record_list['created_at'] = df_plan_record_list['created_at'].astype(np.datetime64)
df_plan_record_list['final_confirm_date'] = df_plan_record_list.final_confirm_date.apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x/1000)))
df_plan_record_list['final_confirm_date'] = df_plan_record_list['final_confirm_date'].astype(np.datetime64)
df_plan_record_list = df_plan_record_list.sort_values(['created_at','final_confirm_date'])
# df_plan_record_list 比 df_trade_list 多了下面 6 列
# ['final_confirm_date', 'total_confirm_amount', 'total_fee', 'bank_name', 'sub_order_list', 'total_confirm_volume']
# 其中，sub_order_list 是详细数据。可能包含 1 ~ n 个对象。每个对象下的 orders 就是具体成交记录
df_plan_record_list.to_excel('02_{0}_plan_order_list.xlsx'.format(global_name), sheet_name=f'{global_name}_交易记录')
# df_plan_record_list[['final_confirm_date', 'total_confirm_amount', 'total_fee', 'bank_name', 'sub_order_list', 'total_confirm_volume']]

D:\develop\python\Anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [24]:
# 处理单笔基金的买入卖出操作
def handle_fund_trades(order_ids):
    # 单只基金无法在 order 页面查看手续费，需要进一步请求
    
    # 并发
    fund_trade_url = 'https://danjuanapp.com/djapi/fund/order/{0}'
    tasks = [grequests.get(fund_trade_url.format(x), headers=headers) for x in order_ids]
    response_list = grequests.map(tasks)
    results = []
    i = 1
    for response in response_list:
        order = response.json()['data']
        s = dict()
        #['id','date','time','code','name','deal_type','nav_unit','nav_acc','volume','deal_money',
        # 'fee','occur_money','account','category1','category2','category3','category_id','note']
        s['id'] = i
        # TODO 这里要判断 15 点前还是 15 点后
        s['date'] = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(int(order['created_at'])/1000))[0:10]
        s['time'] = '15:00:00'
        s['code'] = order['fd_code']
        s['name'] = order['fd_name']
        s['deal_type'] = order['action_desc']
        s['volume'] = order['confirm_volume']
        s['deal_money'] = order['confirm_amount']
        confirm_infos = order['confirm_infos']
        fee = 0.0
        nav_unit = 0.0
        if len(confirm_infos) > 0:
            infos = confirm_infos[0]
            if len(infos) > 0:
                for i in range(len(infos)-1, 0, -1):
                    info = infos[i]
                    if u'手续费' in info:
                        # 手续费,0.04元
                        feeStr = info.replace('手续费,','').replace('元','')
                        fee = round(float(feeStr),2)
                    if u'确认净值' in info:
                        navStr = info.replace('确认净值,','').replace('元','')
                        nav_unit = round(float(navStr),4)
        s['nav_unit'] = nav_unit
        s['nav_acc'] = nav_unit
        s['fee'] = fee
        opType = s['deal_type']
        if opType == '买入' or opType == '转换':
            occur_money = round(s['deal_money'] + fee, 2)
        elif opType == '卖出':
            occur_money = round(s['deal_money'] - fee, 2)
        elif opType == '分红':
            occur_money = round(s['deal_money'], 2)
        s['occur_money'] = occur_money
        s['account'] = '{0}_蛋卷'.format(global_name)
#         s['note'] = 螺丝钉指数基金组合_买入_orderid=2198590579952573614
        s['note'] = order['fd_name'] + '_' + order['action_desc'] + '_orderid=' + order['order_id']
        results.append(pd.Series(s))
        i += 1
    df_fund_trades = pd.DataFrame(results)
    # 补充三级分类
    df_fund_trades = pd.merge(df_fund_trades, df_category, left_on='code', right_on='基金代码', how='left')
    df_fund_trades = df_fund_trades.rename(columns={'一级分类': 'category1', '二级分类': 'category2', '三级分类': 'category3', '分类ID': 'category_id'})
    df_fund_trades = df_fund_trades[['id','date','time','code','name','deal_type','nav_unit','nav_acc','volume','deal_money', 'fee','occur_money','account','category1','category2','category3','category_id','note']]
    
    return df_fund_trades

# 屏蔽货币基金
df_fund_record_list = df_fund_record_list[~df_fund_record_list.name.str.contains('货币')]
if len(df_fund_record_list) > 0:
    df_fund_output_list = handle_fund_trades(df_fund_record_list['order_id'].tolist())
else:
    print('没有单独基金交易')
    df_fund_output_list = pd.DataFrame()
# df_fund_output_list.to_excel('02_{0}_fund_order_list.xlsx'.format(global_name), sheet_name=f'{global_name}_交易记录')

没有单独基金交易


In [25]:
# 每一条成交记录原始表
record_lists = []
for sub_order in df_plan_record_list['sub_order_list']:
    # 如果是转换操作，则 sub_order 就有两个对象
    for i in range(len(sub_order)):
        # 每一个 sub_order 下，每只基金的真实成交列表 orders
        orders = sub_order[i]['orders']
        for record in orders:
            record_lists.append(pd.Series(record))

# 生成最原始的成交记录列表
df_raw = pd.DataFrame(record_lists)
df_raw.index.name = 'id'
df_raw['ts'] = df_raw.ts.apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x/1000)))
df_raw['ts'] = df_raw['ts'].astype(np.datetime64)
df_raw['confirm_ts'] = df_raw.confirm_ts.apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x/1000)))
df_raw['confirm_ts'] = df_raw['confirm_ts'].astype(np.datetime64)
df_raw = df_raw.sort_values(['ts','confirm_ts'])
df_raw.to_excel('03_{0}_all_record_list.xlsx'.format(global_name), sheet_name=f'{global_name}_交易记录')
# df_raw

In [26]:
# ['买入', '卖出', '分红', '转换', '组合转出', '组合转入']
df_raw.action_desc.unique()
# 货币基金转换
df_money_fund_exchange = df_raw[df_raw.action_desc.isin(['转换'])]
# 投资组合内部转换
df_plan_exchange = df_raw[df_raw.action_desc.isin(['组合转出', '组合转入'])]
# 正常买入卖出交易
df_normal = df_raw[df_raw.action_desc.isin(['买入', '卖出', '分红'])]

In [27]:
def deal_type_calc(x):
    if x['action_desc'] == '组合转入':
        return '买入'
    elif x['action_desc'] == '组合转出':
        return '卖出'
    else:
        return x['action_desc']
    
def nav_unit_calc(x):
    # 后面会根据数据库和 confirm_ts 进行修改的，这里只是暂存
    volume = x['confirm_volume']
    money = x['confirm_amount']
    fee = x['fee']
    nav_unit = 1.0
    if x['action_desc'] == '买入':
        nav_unit = round((money - fee) / volume, 4)
    elif x['action_desc'] == '卖出':
        nav_unit = round((money + fee) / volume, 4)
    elif x['action_desc'] == '分红':
        nav_unit = 1.0
    return nav_unit

def occur_money_calc(x):
    # 后面会根据数据库和 confirm_ts 进行修改的，这里只是暂存
    money = x['confirm_amount']
    fee = x['fee']
    if x['action_desc'] in ['买入', '转换']:
        occur_money = round(money + fee, 4)
    elif x['action_desc'] == '卖出':
        occur_money = round(money - fee, 4)
    elif x['action_desc'] == '分红':
        occur_money = money
    return occur_money

In [28]:
# 调整输出

# 买入、卖出、分红
df_temp = df_normal.copy()
print(len(df_temp))
df_temp['id'] = df_normal.reset_index().index + 1
df_temp['date'] = df_temp['ts'].apply(lambda x: str(x)[0:10])
df_temp['time'] = '15:00:00'
df_temp['code'] = df_temp['fd_code']
df_temp['name'] = df_temp['fd_name']
df_temp['deal_type'] = df_temp['action_desc']
df_temp['volume'] = np.abs(df_temp['confirm_volume'])
df_temp['fee'] = df_temp['fee']
df_temp['nav_unit'] = df_temp.apply(nav_unit_calc, axis=1)
df_temp['nav_acc'] = df_temp['nav_unit']
df_temp['deal_money'] = df_temp['confirm_amount']
df_temp['occur_money'] = df_temp.apply(occur_money_calc, axis=1)

df_temp['account'] = '{0}_蛋卷'.format(global_name)
# # 补充一二三级分类
df_temp = pd.merge(df_temp, df_category, left_on='fd_code', right_on='基金代码', how='left')
df_temp = df_temp.rename(columns={'一级分类': 'category1', '二级分类': 'category2', '三级分类': 'category3', '分类ID': 'category_id'})
df_temp['note'] = df_temp['plan_name'] + '_' + df_temp['deal_type'] + '_orderid=' + df_temp['order_id']

df_temp = df_temp[['id','date','time','code','name','deal_type','nav_unit','nav_acc','volume','deal_money','fee','occur_money','account','category1','category2','category3','category_id','note']]
# df_temp.to_excel('04_{0}_normal_record_list.xlsx'.format(global_name), sheet_name=f'{global_name}_交易记录')

540


In [29]:
# 货币基金转换
df_money_temp = df_money_fund_exchange.copy()
print(len(df_money_temp))
if len(df_money_temp) > 0:
    df_money_temp['id'] = df_money_temp.reset_index().index + 1
    df_money_temp['date'] = df_money_temp['ts'].apply(lambda x: str(x)[0:10])
    df_money_temp['time'] = '15:00:00'
    df_money_temp['code'] = df_money_temp['target_fd_code']
    df_money_temp['name'] = df_money_temp['target_fd_name']
    # 目前的基金转换，我只做过南方天天利货币B转指数基金的操作，以后将尽量避免这种繁琐的交易记录
    df_money_temp['action_desc'] = '买入'
    df_money_temp['deal_type'] = df_money_temp['action_desc']
    df_money_temp['volume'] = np.abs(df_money_temp['confirm_volume'])
    df_money_temp['fee'] = df_money_temp['fee']
    df_money_temp['nav_unit'] = df_money_temp.apply(nav_unit_calc, axis=1)
    df_money_temp['nav_acc'] = df_money_temp['nav_unit']
    df_money_temp['deal_money'] = df_money_temp['confirm_amount']
    df_money_temp['occur_money'] = df_money_temp.apply(occur_money_calc, axis=1)

    df_money_temp['account'] = '{0}_蛋卷'.format(global_name)
    # # 补充一二三级分类
    df_money_temp = pd.merge(df_money_temp, df_category, left_on='target_fd_code', right_on='基金代码', how='left')
    df_money_temp = df_money_temp.rename(columns={'一级分类': 'category1', '二级分类': 'category2', '三级分类': 'category3', '分类ID': 'category_id'})
    df_money_temp['note'] = df_money_temp['fd_name'] + '_转换_' + df_money_temp['target_fd_name'] + '_orderid=' + df_money_temp['order_id']

    df_money_temp = df_money_temp[['id','date','time','code','name','deal_type','nav_unit','nav_acc','volume','deal_money','fee','occur_money','account','category1','category2','category3','category_id','note']]
# df_money_temp.to_excel('04_{0}_money_fund_exchange_record_list.xlsx'.format(global_name), sheet_name=f'{global_name}_交易记录')
# df_money_temp

0


In [30]:
# 组合内部转换
df_plan_exchange_temp = df_plan_exchange.copy()
print(len(df_plan_exchange_temp))
df_plan_exchange_temp['id'] = df_plan_exchange_temp.reset_index().index + 1
df_plan_exchange_temp['date'] = df_plan_exchange_temp['ts'].apply(lambda x: str(x)[0:10])
df_plan_exchange_temp['time'] = '15:00:00'
df_plan_exchange_temp['code'] = df_plan_exchange_temp['fd_code']
df_plan_exchange_temp['name'] = df_plan_exchange_temp['fd_name']
# 目前的基金转换，我只做过南方天天利货币B转指数基金的操作，以后将尽量避免这种繁琐的交易记录
df_plan_exchange_temp['action_desc_exchange'] = df_plan_exchange_temp['action_desc']
df_plan_exchange_temp['action_desc'] = df_plan_exchange_temp.apply(deal_type_calc, axis=1)
df_plan_exchange_temp['deal_type'] = df_plan_exchange_temp['action_desc']
df_plan_exchange_temp['volume'] = np.abs(df_plan_exchange_temp['confirm_volume'])
df_plan_exchange_temp['fee'] = df_plan_exchange_temp['fee']
df_plan_exchange_temp['nav_unit'] = df_plan_exchange_temp.apply(nav_unit_calc, axis=1)
df_plan_exchange_temp['nav_acc'] = df_plan_exchange_temp['nav_unit']
df_plan_exchange_temp['deal_money'] = df_plan_exchange_temp['confirm_amount']
df_plan_exchange_temp['occur_money'] = df_plan_exchange_temp.apply(occur_money_calc, axis=1)

df_plan_exchange_temp['account'] = '{0}_蛋卷'.format(global_name)
# # 补充一二三级分类
df_plan_exchange_temp = pd.merge(df_plan_exchange_temp, df_category, left_on='fd_code', right_on='基金代码', how='left')
df_plan_exchange_temp = df_plan_exchange_temp.rename(columns={'一级分类': 'category1', '二级分类': 'category2', '三级分类': 'category3', '分类ID': 'category_id'})
df_plan_exchange_temp['note'] = df_plan_exchange_temp['plan_name'] + '_' + df_plan_exchange_temp['action_desc_exchange'] + '_' + df_plan_exchange_temp['target_fd_name'] +  '_orderid=' + df_plan_exchange_temp['order_id']

df_plan_exchange_temp = df_plan_exchange_temp[['id','date','time','code','name','deal_type','nav_unit','nav_acc','volume','deal_money','fee','occur_money','account','category1','category2','category3','category_id','note']]
# df_plan_exchange_temp.to_excel('04_{0}_plan_exchange_record_list.xlsx'.format(global_name), sheet_name=f'{global_name}_交易记录')
# df_plan_exchange_temp

66


In [32]:
# 最终合并
all = []
if len(df_temp) > 0:
    all.append(df_temp)
if len(df_money_temp) > 0:
    all.append(df_money_temp)
if len(df_plan_exchange_temp) > 0:
    all.append(df_plan_exchange_temp)
# 如果有基金交易，那也算上吧
if len(df_fund_output_list) > 0:
    all.append(df_fund_output_list)
df_output = pd.concat(all,ignore_index=True, sort=False)
df_output = df_output.sort_values(['date', 'code'])
df_output = df_output.reset_index(drop=True)
df_output.id = df_output.index + 1
df_output.to_excel('04_{0}_蛋卷.xlsx'.format(global_name), sheet_name=f'{global_name}_交易记录')